# Linear models with CNN features

## why convbattle2 dataset ?
convbattle fas major defaults :
1. some pict were representing anything but shoes
1. some shoes were really too small on standing people pict to be recognized
1. but, moreover, the dataset is hugely biased, validation set comes from pinterest, train from zappos and google

how to build convbattle2 ?
1. removing bad pict
1. shuffling datasets

In [1]:
from __future__ import division, print_function
# Rather than importing everything manually, we'll make things easy
#   and load them all in utils.py, and just import them from there.
%matplotlib inline
from importlib import reload  # Python 3
import utils; reload(utils)
from utils import *

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
Using TensorFlow backend.


In [2]:
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
from utils import plots, get_batches, plot_confusion_matrix, get_data

In [3]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

In [4]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [5]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [6]:
def plots_idx(idx, titles=None):
    plots([image.load_img(path + 'valid/' + filenames[i]) for i in idx], titles=titles)

In [7]:
#shape dim th patch
from keras import backend as K
K.set_image_dim_ordering('th')

## Train linear model on predictions

In [8]:
path = "convbattle2/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [10]:
#batch_size=256 => KO
batch_size=64
#batch_size=64
#batch_size=4

We need to start with our VGG 16 model, since we'll be using its predictions and features.

In [11]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

In [12]:
# Use batch size of 1 since we're just doing preprocessing on the CPU
val_batches = get_batches(path+'valid', shuffle=False, batch_size=1)
batches = get_batches(path+'train', shuffle=False, batch_size=1)

Found 551 images belonging to 2 classes.
Found 2213 images belonging to 2 classes.


In [13]:
#usefull for pict plotting
filenames = val_batches.filenames

We have provided a simple function that joins the arrays from all the batches - let's use this to grab the training and validation data:

#### skip next steps


In [ ]:
val_data = get_data(path+'valid')

In [ ]:
trn_data = get_data(path+'train')

In [ ]:
trn_data.shape

In [ ]:
save_array(model_path+ 'train_data.bc', trn_data)
save_array(model_path + 'valid_data.bc', val_data)

We can load our training and validation data later without recalculating them:

##### as we have saved data.bc, we can skip get_batches and get_data

In [14]:
#trn_data = load_array(model_path+'train_data.bc')
#val_data = load_array(model_path+'valid_data.bc')

#dim shape patch roll axis from array_shape=(None, 3, 224, 224) to array_shape=(2378, 224, 224, 3)
#TF complient predictions

trn_data = np.rollaxis(load_array(model_path+'train_data.bc'), 3, 1)
val_data = np.rollaxis(load_array(model_path+'valid_data.bc'), 3, 1)

In [15]:
val_data.shape

(551, 3, 224, 224)

In [16]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

## Retrain last layer's linear model

In [17]:
model.pop()
for layer in model.layers: layer.trainable=False

In [18]:
model.add(Dense(2, activation='softmax'))

In [19]:
layers = model.layers
# Get the index of the first dense layer...
first_dense_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense][0]
# ...and set this and all subsequent layers to trainable
for layer in layers[first_dense_idx:]: layer.trainable=True

In [ ]:
model.summary()

In [20]:
opt = RMSprop(lr=0.0003)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#batches = get_batches(path+'train', shuffle=True, batch_size=1)
#vgg.finetune(batches)

In [ ]:
#??vgg.finetune

In [21]:
gen=image.ImageDataGenerator()
batches = gen.flow(trn_data, trn_labels, batch_size=batch_size, shuffle=True)
val_batches = gen.flow(val_data, val_labels, batch_size=batch_size, shuffle=False)
steps_per_epoch = int(np.ceil(batches.n/batch_size))
validation_steps = int(np.ceil(val_batches.n/batch_size))

In [22]:
print(trn_labels.shape)
print(val_labels.shape)

(2213, 2)
(551, 2)


In [23]:
print("batch size:" + str(batch_size))
print("calculated steps_per_epoch:" + str(steps_per_epoch) + " | calculated validation_steps:" + str(validation_steps))

batch size:64
calculated steps_per_epoch:35 | calculated validation_steps:9


We'll define a simple function for fitting models, just to save a little typing...

In [24]:
def fit_model(model, batches, val_batches, nb_epoch=1):
    model.fit_generator(batches, steps_per_epoch=steps_per_epoch, epochs=nb_epoch, 
                        validation_data=val_batches, validation_steps=validation_steps)

In [25]:
model.save_weights(model_path+'finetune.studyg.epoch0.h5')

In [30]:
model.load_weights(model_path+'finetune.studyg.epoch0.h5')

In [32]:
opt = RMSprop(lr=0.1)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
#K.set_value(opt.lr, 0.1)
fit_model(model, batches, val_batches, 1)

Epoch 1/1
35/35 [==============================] - 54s - loss: 6.7018 - acc: 0.5842 - val_loss: 6.8451 - val_acc: 0.5753


In [ ]:
model.evaluate(val_data, val_labels)

We can look at the earlier prediction examples visualizations by redefining *probs* and *preds* and re-using our earlier code.

In [ ]:
preds = model.predict_classes(val_data, batch_size=batch_size)
probs = model.predict_proba(val_data, batch_size=batch_size)[:,0]
probs[:8]

In [ ]:
cm = confusion_matrix(val_classes, preds)

In [ ]:
plot_confusion_matrix(cm, {'converse':0, 'newbalance':1})

#### note : when overfitting, newbalance are more predicted as converse

In [ ]:
#print(filenames)
n_view = 6

#1. A few correct labels at random
correct = np.where(preds==val_labels[:,1])[0]
idx = permutation(correct)[:n_view]
plots_idx(idx, probs[idx])

In [ ]:
#2. A few incorrect labels at random
incorrect = np.where(preds!=val_labels[:,1])[0]
idx = permutation(incorrect)[:n_view]
plots_idx(idx, probs[idx])

In [ ]:
#3. The images we most confident were CONVERSE, and are actually CONVERSE
correct_converse = np.where((preds==0) & (preds==val_labels[:,1]))[0]
most_correct_converse = np.argsort(probs[correct_converse])[::-1][:n_view]
plots_idx(correct_converse[most_correct_converse], probs[correct_converse][most_correct_converse])

# as above, but NEW BALANCE
correct_newb = np.where((preds==1) & (preds==val_labels[:,1]))[0]
most_correct_newb = np.argsort(probs[correct_newb])[:n_view]
plots_idx(correct_newb[most_correct_newb], 1-probs[correct_newb][most_correct_newb])

In [ ]:
#4. The images we were most confident were CONVERSE, but are actually NEWBALANCE
incorrect_converse = np.where((preds==0) & (preds!=val_labels[:,1]))[0]
most_incorrect_converse = np.argsort(probs[incorrect_converse])[::-1][:n_view]
plots_idx(incorrect_converse[most_incorrect_converse], probs[incorrect_converse][most_incorrect_converse])

# as above, but NEW BALANCE
incorrect_newb = np.where((preds==1) & (preds!=val_labels[:,1]))[0]
most_incorrect_newb = np.argsort(probs[incorrect_newb])[::-1][:n_view]
plots_idx(incorrect_newb[most_incorrect_newb], 1-probs[incorrect_newb][most_incorrect_newb])

##### maybe linked to unbalanced confusion matrix, worst predict on new balance got 50% probs. no high level % = quite discriminating

In [ ]:
#5. The most uncertain labels (ie those with probability closest to 0.5).
most_uncertain = np.argsort(np.abs(probs-0.5))
plots_idx(most_uncertain[:n_view], probs[most_uncertain])